# Convolution Neural Network (full one)

## CNN (how we will build it)

Image

🠋

Convolution

🠋

ReLU

🠋

Pooling

🠋

Convolution

🠋

ReLU

🠋

Pooling

🠋

Flatten

🠋

Dense

🠋

Outcome


Now that we understand it, let's get down to code.

## Code